In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import csv
from time import sleep
import ast
import pandas as pd
import urllib.request
import os

def crawling(keyword):
    dir_driver="./chromedriver"
    driver=webdriver.Chrome(dir_driver)
    sleep(0.01)
    base_url="https://www.yna.co.kr/search/index?query={}".format(keyword)
    driver.get(base_url)
    object = bs(driver.page_source, 'html.parser')

    # 제목 크롤링
    subjects = object.find_all('span', {'class':'tt2'})
    subjects = [subject.text for subject in subjects]

    # 내용 크롤링
    contents = object.find_all('span', {'class':'cts'})
    contents = [content.text for content in contents]

    # 이미지 크롤링
    images = object.find('div', {'class':'cts_atclst'})
    images = images.find_all('img')
    images = ["https:" + image.get('src') for image in images]
    for i in range(len(images)):
        urllib.request.urlretrieve(images[i], "./images/" + str(i) + ".jpg")

    # 태그 크롤링
    tags = list()
    for i in range(1, 6):
        driver.find_element_by_xpath('//*[@id="article_list"]/div[2]/ul/li[{}]/a'.format(i)).click()
        sleep(0.1)
        object = bs(driver.page_source, 'html.parser')
        temp_tags = object.select('#articleWrap > div.content01.scroll-article-zone01 > div > div > div.keyword-zone > ul > li > a')
        temp_tags = [temp_tag.text.replace("#", "") for temp_tag in temp_tags]
        tags.append(temp_tags)
        driver.back()
        sleep(0.01)

    data = {"subjects" : subjects, "contents":contents, "tags":tags, "images":images}
    driver.close()
    return data

In [2]:
import psycopg2
from datetime import datetime, date
def connect_db():
    host = "localhost"
    dbname = "postgres"
    user = "postgres"
    password = "postgres"
    sslmode = "allow"

    conn_string = "host={0} user={1} dbname={2} password={3} sslmode={4}".format(host, user, dbname, password, sslmode)
    conn = psycopg2.connect(conn_string)
    cursor = conn.cursor()
    return conn, cursor

def insert_data(data, cursor):
    cursor.execute("SELECT news_no FROM news_contents;")
    rows = cursor.fetchall()
    news_no = max([row[0] for row in rows]) if len(rows)!=0 else 0

    cursor.execute("SELECT seq FROM news_tag;")
    rows = cursor.fetchall()
    seq = max([row[0] for row in rows]) if len(rows)!=0 else 0

    for i in range(len(data['subjects'])):
        news_no += 1
        title = "'{}'".format(data['subjects'][i].replace("'", "''", 10000))
        content = "'{}'".format(data['contents'][i].replace("'", "''", 10000))
        mem_no = "'김가영'"
        crawl_dt = "'{}'".format(datetime.today().strftime("%Y-%m-%d"))
        crawl_tm = "'{}'".format(datetime.today().strftime("%H:%M:%S"))
        cursor.execute("INSERT INTO news_contents (news_no, title, content, mem_no, crawl_dt, crawl_tm) VALUES ({}, {}, {}, {}, {}, {});".format(news_no, title, content, mem_no, crawl_dt, crawl_tm))
        for tag in data['tags'][i]:
            seq += 1
            tag = "'{}'".format(tag)
            del_yn = "'N'"
            reg_mem_no = "'김가영'"
            reg_dt = "'{}'".format(datetime.today().strftime("%Y-%m-%d"))
            reg_tm = "'{}'".format(datetime.today().strftime("%H:%M:%S"))
            cursor.execute("INSERT INTO news_tag (news_no, seq, tag, del_yn, reg_mem_no, reg_dt, reg_tm) VALUES ({}, {}, {}, {}, {}, {}, {});".format(news_no, seq, tag, del_yn, reg_mem_no, reg_dt, reg_tm))

def save_data(conn, cursor):
    conn.commit()
    cursor.close()
    conn.close()

In [3]:
def execute():
    num = int(input("단어의 개수 => "))
    keyword_list = list()
    
    for _ in range(num):
        temp_input = input("단어 => ")
        keyword_list.append(temp_input)
    
    for keyword in keyword_list:
        data = crawling(keyword)
        conn, cursor = connect_db()
        insert_data(data, cursor)
        save_data(conn, cursor)
    print("==========\n완료하였습니다.")
    
def simple_execute(keyword):
    data = crawling(keyword)
    conn, cursor = connect_db()
    insert_data(data, cursor)
    save_data(conn, cursor)
    print("==========\n완료하였습니다.")

## 특정 시간에 실행

In [4]:
#!pip install schedule

In [ ]:
import schedule
import time

job = schedule.every().day.at("09:00:00").do(simple_execute, "크루즈")

while True:
    schedule.run_pending()
    time.sleep(1)

완료하였습니다.
